In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from pathlib import Path
import pickle

%load_ext autoreload
%autoreload 2
from base import datahandler, prediction_models, evaluation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
ew_localizer = tf.keras.models.load_model("models/ew_localizer.hdf5")
ns_localizer = tf.keras.models.load_model("models/ns_localizer.hdf5")
ew_ns_classifier = tf.keras.models.load_model("models/ew_ns_classifier.hdf5")

2024-01-12 11:18:59.289420: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1c:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-12 11:18:59.313045: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1c:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-12 11:18:59.313116: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1c:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-12 11:18:59.316094: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1c:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-12 11:18:59.316172: I external/local_xla/xla/stream_executor

In [4]:
# load some validation data
challenge_data_dir = Path('dataset/phase_1/')
data_dir = challenge_data_dir / "train"
labels_dir = challenge_data_dir / 'train_labels.csv'

split_dataframes = datahandler.load_and_prepare_dataframes(data_dir, labels_dir)
# train_keys = list(split_dataframes.keys())[150:]
# val_keys = list(split_dataframes.keys())[150:200]
# some_dataframes = {df_k : split_dataframes[df_k] for df_k in val_keys}

input_features_reduced = ['Eccentricity', 'Semimajor Axis (m)', 'Inclination (deg)', 'RAAN (deg)',
       'Argument of Periapsis (deg)', 'True Anomaly (deg)', 'Latitude (deg)',
       'Longitude (deg)', 'Altitude (m)']
ew_input_features = ['Eccentricity', 'Semimajor Axis (m)', 'Argument of Periapsis (deg)', 'Longitude (deg)', 'Altitude (m)']
ns_input_features = ['Eccentricity', 'Semimajor Axis (m)', 'RAAN (deg)', 'Inclination (deg)', 'Latitude (deg)', 'Altitude (m)']



In [5]:
challenge_data_dir = Path('dataset/phase_1/')
data_dir = challenge_data_dir / "train"
split_dataframes = datahandler.load_and_prepare_dataframes(data_dir, None)
ew_input_features = ['Eccentricity', 'Semimajor Axis (m)', 'Argument of Periapsis (deg)', 'Longitude (deg)', 'Altitude (m)']

#some_dataframes = {k:split_dataframes[k] for k in ['104','105']}
# EW
ds_gen = datahandler.DatasetGenerator(split_df=split_dataframes,
                                      input_features=ew_input_features,
                                      label_features=[],
                                      train_val_split=0.8,
                                      stride=1,
                                      input_stride=1,
                                      padding='none',
                                      input_history_steps=20,
                                      input_future_steps=20,
                                      seed=69)
train_ds, test_ds = ds_gen.get_datasets(128, label_features=[], shuffle=False, keep_identifier=True)

inputs = np.concatenate([element for element in test_ds.map(lambda x,z: x).as_numpy_iterator()])
identifiers_ew = np.concatenate([element for element in test_ds.map(lambda x,z: z).as_numpy_iterator()])

ew_localizer = tf.keras.models.load_model('models/ew_localizer.hdf5')
preds_ew = ew_localizer.predict(inputs, verbose=2)
preds_ew_argmax = np.argmax(preds_ew, axis=1)

df_ew = pd.DataFrame(np.concatenate([identifiers_ew.reshape(-1,2)], axis=1), columns=['ObjectID', 'TimeIndex'], dtype=np.int32)
df_ew[f'Location_EW'] = preds_ew_argmax

print(df_ew["ObjectID"].unique())
print(df_ew.loc[(df_ew['ObjectID']==104) & (df_ew['Location_EW'] == 1)].head(30))


loading data without labels
=========================Creating Dataset=========================
Seed: 69
nTrain: 200 nVal: 50 (0.80)
Padding: False
Scaling: True 
Horizons: 20-20 @ stride 1
Sin-Transformed features: ['Argument of Periapsis (deg)']
No Labels
=========================Finished Dataset=========================
3403/3403 - 8s - 8s/epoch - 2ms/step
[ 10 104 114 115 116 119 121 123 137 139 148  15 160 161 167 171 176 184
 185 193 198 199  21 211 212 221 222 227 230 233 235 238 241 245  26  30
  36  45  47  50  52  60  61  66  67  73  84  87  88  91]
      ObjectID  TimeIndex  Location_EW
2269       104         98            1
2270       104         99            1
2271       104        100            1
2288       104        117            1
2289       104        118            1
2290       104        119            1
2291       104        120            1


In [6]:
# EW
ds_gen = datahandler.DatasetGenerator(split_df=split_dataframes,
                                      input_features=ew_input_features,
                                      label_features=[],
                                      train_val_split=0.8,
                                      stride=1,
                                      input_stride=1,
                                      padding=False,
                                      input_history_steps=20,
                                      input_future_steps=20,
                                      seed=69)
train_ds, test_ds = ds_gen.get_datasets(128, label_features=[], shuffle=False, keep_identifier=True)
pickle.dump(ds_gen.scaler, open('submission/models/EW_localizer_scaler.pkl', 'wb'))

inputs = np.concatenate([element for element in test_ds.map(lambda x,z: x).as_numpy_iterator()])
identifiers_ew = np.concatenate([element for element in test_ds.map(lambda x,z: z).as_numpy_iterator()])

ew_localizer = tf.keras.models.load_model('models/ew_localizer.hdf5')
preds_ew = ew_localizer.predict(inputs, verbose=2)
preds_ew_argmax = np.argmax(preds_ew, axis=1)

# NS-Localization
ds_gen = datahandler.DatasetGenerator(split_df=split_dataframes,
                                      input_features=ns_input_features,
                                      label_features=[],
                                      train_val_split=0.8,
                                      stride=1,
                                      input_stride=4,
                                      padding='none',
                                      input_history_steps=80,
                                      input_future_steps=80,
                                      seed=69)
train_ds, test_ds = ds_gen.get_datasets(128, label_features=[], shuffle=False, keep_identifier=True)
pickle.dump(ds_gen.scaler, open('submission/models/NS_localizer_scaler.pkl', 'wb'))


inputs = np.concatenate([element for element in test_ds.map(lambda x,z: x).as_numpy_iterator()])
identifiers_ns = np.concatenate([element for element in test_ds.map(lambda x,z: z).as_numpy_iterator()])

ns_localizer = tf.keras.models.load_model('models/ns_localizer.hdf5')
preds_ns = ns_localizer.predict(inputs, verbose=2)
preds_ns_argmax = np.argmax(preds_ns, axis=1)

# Localization cleanup
df_ew = pd.DataFrame(np.concatenate([identifiers_ew.reshape(-1,2)], axis=1), columns=['ObjectID', 'TimeIndex'], dtype=np.int32)
df_ew[f'Location_EW'] = preds_ew_argmax
df_ns = pd.DataFrame(np.concatenate([identifiers_ns.reshape(-1,2)], axis=1), columns=['ObjectID', 'TimeIndex'], dtype=np.int32)
df_ns[f'Location_NS'] = preds_ns_argmax

# remove consecutives detections
print(df_ew.loc[(df_ew['ObjectID']==104) & (df_ew['Location_EW'] == 1)].head(30))
df_ew = df_ew.sort_values(['ObjectID', 'TimeIndex']).reset_index(drop=True)
df_ew = df_ew.loc[(df_ew['Location_EW']==1)]
df_ew['consecutive'] = (df_ew['TimeIndex'] - df_ew['TimeIndex'].shift(1) != 1).cumsum()
df_ew=df_ew.groupby('consecutive').apply(lambda sub_df: sub_df.iloc[int(len(sub_df)/2), :]).reset_index(drop=True).drop(columns=['consecutive'])

df_ns = df_ns.sort_values(['ObjectID', 'TimeIndex']).reset_index(drop=True)
df_ns = df_ns.loc[(df_ns['Location_NS']==1)]
df_ns['consecutive'] = (df_ns['TimeIndex'] - df_ns['TimeIndex'].shift(1) != 1).cumsum()
df_ns=df_ns.groupby('consecutive').apply(lambda sub_df: sub_df.iloc[int(len(sub_df)/2), :]).reset_index(drop=True).drop(columns=['consecutive'])
    
print(df_ew.loc[(df_ew['ObjectID']==104) & (df_ew['Location_EW'] == 1)].head(30))
#df = df_ew.merge(df_ns, how='outer', on = ['ObjectID', 'TimeIndex']).fillna(0).astype(np.int32)

# TEMPORARY
df = df_ew
df_ew['Location_NS'] = 0

# add initial node prediction
object_ids = list(map(int, list(split_dataframes.keys())))
for obj_id in object_ids:
    df = df.sort_index()
    df.loc[-1] = [int(obj_id), 0, 1, 1] # objid, timeindex, Location_EW, Location_NS
    df.index = df.index + 1
    df = df.sort_index()

df_locs = df.loc[(df['Location_EW'] == 1) | (df['Location_NS'] == 1)]
df_locs = df_locs.sort_values(['ObjectID', 'TimeIndex']).reset_index(drop=True)

print(len(df_locs))
print(df_locs.head(20))

=========================Creating Dataset=========================
Seed: 69
nTrain: 200 nVal: 50 (0.80)
Padding: False
Scaling: True 
Horizons: 20-20 @ stride 1
Sin-Transformed features: ['Argument of Periapsis (deg)']
No Labels
=========================Finished Dataset=========================
3403/3403 - 8s - 8s/epoch - 2ms/step
=========================Creating Dataset=========================
Seed: 69
nTrain: 200 nVal: 50 (0.80)
Padding: False
Scaling: True 
Horizons: 80-80 @ stride 4
Sin-Transformed features: []
No Labels
=========================Finished Dataset=========================


2024-01-12 11:19:56.657215: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


3403/3403 - 11s - 11s/epoch - 3ms/step
      ObjectID  TimeIndex  Location_EW
2269       104         98            1
2270       104         99            1
2271       104        100            1
2288       104        117            1
2289       104        118            1
2290       104        119            1
2291       104        120            1
   ObjectID  TimeIndex  Location_EW
0       104         99            1
1       104        119            1
280
    ObjectID  TimeIndex  Location_EW  Location_NS
0          1          0            1            1
1          2          0            1            1
2          3          0            1            1
3          4          0            1            1
4          5          0            1            1
5          6          0            1            1
6          7          0            1            1
7          8          0            1            1
8          9          0            1            1
9         10          0            1 

In [7]:
# now we need to get label predictions on those locations
ds_gen = datahandler.DatasetGenerator(split_df=split_dataframes,
                                      input_features=input_features_reduced,
                                      label_features=[],
                                      train_val_split=0.8,
                                      stride=1,
                                      input_stride=2, #!
                                      padding='none',
                                      input_history_steps=40,
                                      input_future_steps=40,
                                      seed=69)
train_ds, test_ds = ds_gen.get_datasets(128, label_features=[], shuffle=False, keep_identifier=True)
pickle.dump(ds_gen.scaler, open('submission/models/classifier_scaler.pkl', 'wb'))


inputs = np.concatenate([element for element in test_ds.map(lambda x,z: x).as_numpy_iterator()])
#labels = np.concatenate([element['EW_Node_Location'] for element in ds.map(lambda x,y,z: y).as_numpy_iterator()])
identifiers = np.concatenate([element for element in test_ds.map(lambda x,z: z).as_numpy_iterator()])


# get predictions
preds = ew_ns_classifier.predict(inputs)
sub_dfs = []
for ft_idx, feature in enumerate(['EW', 'NS']):
    sub_df = pd.DataFrame(np.concatenate([identifiers.reshape(-1,2)], axis=1), columns=['ObjectID', 'TimeIndex'], dtype=np.int32)
    sub_df['Direction'] = feature
    preds_argmax = np.argmax(preds[ft_idx], axis=1)
    sub_df[f'{feature}_Decoded'] = ds_gen.combined_label_encoder.inverse_transform(preds_argmax)
    sub_df[['Node', 'Type']] = sub_df[f'{feature}_Decoded'].str.split('-', expand=True)
    sub_df = sub_df.drop([f'{feature}_Decoded'], axis='columns')
    sub_dfs.append(sub_df)

# Add direction column
df_classes = pd.concat(sub_dfs).sort_values(['ObjectID', 'TimeIndex']).reset_index(drop=True)
# For timeindex 0, the node is always SS
df_classes.loc[df_classes['TimeIndex'] == 0, 'Node'] = 'SS'

print(df_classes.head(5))

=========================Creating Dataset=========================
Seed: 69
nTrain: 200 nVal: 50 (0.80)
Padding: True
Scaling: True 
Horizons: 40-40 @ stride 2
Sin-Transformed features: ['True Anomaly (deg)', 'Argument of Periapsis (deg)']
No Labels
=========================Finished Dataset=========================
2352/3403 [===================>..........] - ETA: 6s

In [ ]:
# Combine the classifications with the node locations
df_merged = df_locs.merge(df_classes, how='left', on = ['ObjectID', 'TimeIndex'])
df_reduced = df_merged[((df_merged['Location_EW'] == 1) & (df_merged['Direction'] == 'EW') | (df_merged['Location_NS'] == 1) & (df_merged['Direction'] == 'NS'))]
df_reduced = df_reduced.drop(['Location_EW'], axis='columns')
df_reduced = df_reduced.drop(['Location_NS'], axis='columns')
print(df_reduced.head(5))

    ObjectID  TimeIndex Direction Node Type
9         10          0        EW   SS   HK
10        10          0        NS   SS   HK
15        15          0        EW   SS   EK
16        15          0        NS   SS   CK
22        21          0        EW   SS   EK


In [ ]:
# Finally, perform the evaluation
ground_truth = pd.read_csv(challenge_data_dir / 'train_labels.csv')
ground_truth = ground_truth[ground_truth['ObjectID'].isin(list(df_reduced["ObjectID"].unique()))].copy()

evaluator = evaluation.NodeDetectionEvaluator(ground_truth, df_reduced)
precision, recall, f2, rmse, total_tp, total_fp, total_fn = evaluator.score()
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F2: {f2:.2f}')
print(f'RMSE: {rmse:.4}')
print(f'TP: {total_tp} FP: {total_fp} FN: {total_fn}')
# 0.77 0.51 0.55 0.29 46 14 444

Precision: 0.66
Recall: 0.72
F2: 0.71
RMSE: 0.199
TP: 101 FP: 51 FN: 39


In [9]:
print(ground_truth.head(50))

     ObjectID  TimeIndex Direction Node Type
8         136          0        EW   SS   NK
9         136          0        NS   SS   NK
10        136       1504        EW   AD   NK
11        136       1511        EW   IK   CK
12        136       2171        ES   ES   ES
16        171          0        EW   SS   CK
17        171          0        NS   SS   CK
18        171        157        EW   ID   NK
19        171        157        NS   ID   NK
20        171        208        EW   AD   NK
21        171       2171        ES   ES   ES
65        211          0        EW   SS   HK
66        211          0        NS   SS   HK
67        211        145        EW   ID   NK
68        211        145        NS   ID   NK
69        211        328        EW   AD   NK
70        211        397        EW   AD   NK
71        211        408        EW   IK   HK
72        211        442        NS   IK   HK
73        211       2183        ES   ES   ES
83         96          0        EW   SS   CK
84        